In [23]:
import json

with open("sample_config.json", "r") as sample_file:
    template = json.load(sample_file)
    
class RefPath:
    def __init__(self, ref_path):
        paths = ref_path.split("#/info/x-pods/")
        parts = paths[1].split("/")
        self.pod_name = parts[0]
        self.container_name = parts[2]

    @property
    def full_path(self):
        return "#/info/x-pods/" + self.pod_name + "/containers/" + self.container_name + "/port"


class Method:
    def __init__(self, path_name, method_name, ref_path, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path
        self.full_method = full_method

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ")"
    
class Pod:
    def __init__(self, pod_name, full_pod):
        self.pod_name = pod_name
        self.full_pod = full_pod

In [32]:
methods = []
for path_name, path in template["paths"].items():
    for method_name, method in path.items():
        ref_path = RefPath(method["x-location"]["$ref"])
        full_method = dict()
        full_method[method_name] = method
        methods.append(Method(path_name, method_name, ref_path, full_method))

pods = []
for pod_name, pod in template["info"]["x-pods"].items():
    full_pod = dict()
    full_pod[pod_name] = pod
    pods.append(Pod(pod_name, full_pod))

In [51]:
import copy
templates = []
for pod in pods:
    sample_template = copy.deepcopy(template)
    pod_methods = {}
    for method in methods:
        if method.ref_path.pod_name == pod.pod_name:
            pod_methods[method.path_name] = method.full_method
    if len(pod_methods):
        sample_template["info"]["x-pods"] = pod.full_pod
        sample_template["paths"] = pod_methods

        templates.append(sample_template)

In [54]:
templates[3]

{'openapi': '3.0.0',
 'info': {'description': 'This is a sample server Petstore server.  You can find out more about     Swagger at [http://swagger.io](http://swagger.io) or on [irc.freenode.net, #swagger](http://swagger.io/irc/).      For this sample, you can use the api key `special-key` to test the authorization     filters.',
  'version': '1.0.0',
  'title': 'Swagger Petstore',
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'x-pods': {'pod4': {'name': 'pod4',
    'metrics': {'CPU': '', 'RAM': ''},
    'containers': {'c1': {'id': 'c1',
      'metrics': {'load': ''},
      'port': 4004}}}}},
 'servers': [{'url': 'https://petstore.swagger.io',
   'description': 'Optional server description, e.g. Main (production) server'}],
 'tags': [{'name': 'pet',
   'description': 'Everything about your Pets',
   'externalDocs': {'description': 'Find out